In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from maritime_traffic_network import MaritimeTrafficNetwork
import folium
import warnings
import sys
import pickle
warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
print(sys.path)

# import modules
import visualize

['/Users/janhendrikwebert/maritime_route_prediction/src/models', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python311.zip', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/lib-dynload', '', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/site-packages', '../visualization', '../features', '../visualization', '../features']


In [3]:
# read data from file
datasize = 'full'
location = 'oslo'
data_date = '202208'
filename = '../../data/processed/'+data_date+'_points_'+location+'_cleaned_meta_'+datasize+'_dualSplit_2.parquet'
#filename = '../../data/processed/202204_points_stavanger_cleaned_meta_full.parquet'
gdf = gpd.read_parquet(filename)

In [4]:
# filter the data according to ship type
#shiptypes = ['Tank', 'Last', 'Passasjer', None]
#gdf = gdf[gdf.skipsgruppe.isin(shiptypes)]

In [5]:
# Transform to desired CRS
# 4326 for WGS 84 (global) // 32632 for UTM 32N (Norway)
crs = 32632  # Coordinate reference system
gdf.to_crs(crs, inplace=True)  # Transformation
gdf.head()

,mmsi,imo_nr,length,lon,lat,sog,cog,true_heading,nav_status,message_nr,bredde,dypgaaende,skipstype,skipsgruppe,fartoynavn,geometry,speed
date_time_utc,,,,,,,,,,,,,,,,,
2022-08-23 14:08:08,205689000_0_2022-08-23 14:08:08,9345362,82,10.7319,58.9092,7.7,352.0,352,0,1,12.5,5.3,General Cargo Ship,Last,FAST HERMAN,POINT (599756.409 6531232.716),3.391128
2022-08-23 14:08:18,205689000_0_2022-08-23 14:08:08,9345362,82,10.7318,58.9095,7.7,351.2,352,0,1,12.5,5.3,General Cargo Ship,Last,FAST HERMAN,POINT (599749.785 6531265.964),3.391128
2022-08-23 14:08:28,205689000_0_2022-08-23 14:08:08,9345362,82,10.7317,58.9099,7.7,351.2,352,0,1,12.5,5.3,General Cargo Ship,Last,FAST HERMAN,POINT (599742.873 6531310.345),4.492851
2022-08-23 14:08:37,205689000_0_2022-08-23 14:08:08,9345362,82,10.7316,58.9102,7.7,351.2,352,0,1,12.5,5.3,General Cargo Ship,Last,FAST HERMAN,POINT (599736.250 6531343.594),3.767918
2022-08-23 14:08:48,205689000_0_2022-08-23 14:08:08,9345362,82,10.7315,58.9105,7.7,351.2,352,0,1,12.5,5.3,General Cargo Ship,Last,FAST HERMAN,POINT (599729.626 6531376.842),3.082842


In [6]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf, crs)
network.get_trajectories_info()

Number of AIS messages: 2173821
Number of trajectories: 7184
Coordinate Reference System (CRS): EPSG:32632


In [7]:
# calculate significant turning points using Douglas Peucker algorithm
tolerance = 10 # DP tolerance parameter 0.0002
network.calc_significant_points_DP(tolerance)

Calculating significant turning points with Douglas Peucker algorithm (tolerance = 10) ...
Number of significant points detected: 181786 (8.36% of AIS messages)
Time elapsed: 2.15 minutes
Adding course over ground before and after each turn ...
Done. Time elapsed: 1.82 minutes


In [8]:
# detect waypoints using spatial clustering
method = 'HDBSCAN'      # 'DBSCAN' , 'HDBSCAN', 'OPTICS'
metric = 'mahalanobis'  # 'euclidean', 'mahalanobis', 'haversine'
min_samples = 20
min_cluster_size = 20
eps = 0
V = np.diag([1, 1, 0.01, 0.01, 1])  # mahalanobis distance parameter matrix V = np.diag([1, 1, 0.01, 0.01, 1e-3])  seems to be good
network.calc_waypoints_clustering(method=method, min_samples=min_samples, min_cluster_size=min_cluster_size,
                                  eps=eps, metric=metric, V=V)

Calculating waypoints with HDBSCAN (min_samples = 20) ...
Distance metric: mahalanobis
816 clusters detected
Time elapsed: 5.33 minutes


In [9]:
# make graph from waypoints
max_distance=20
max_angle=45
network.make_graph_from_waypoints(max_distance=max_distance, max_angle=max_angle)

Constructing maritime traffic network graph from waypoints and trajectories...
Progress: 10%...20%...30%...40%...50%...60%...70%...80%...90%...Done!
------------------------
Unpruned Graph:
Number of nodes: 816 (4 isolated)
Number of edges: 3945
Network is (weakly) connected: False
------------------------
Time elapsed: 8.13 minutes


In [10]:
model = data_date+'_waypoints_DP' + str(tolerance) + '_' + method + str(min_samples) +'_'+location+'_'+datasize+'_UTM_filtered'
merge_stops = True
merge_stops_speed = 2
pruning = 1
network.merge_stop_points(max_speed=merge_stops_speed)
network.prune_graph(pruning)

Pruning...
------------------------
Pruned Graph:
Number of nodes: 816 (4 isolated)
Number of edges: 3860
------------------------


In [11]:
# save hyperparameters
params = {
    'Data':filename,
    'DP_tolerance':tolerance,
    'clustering_method':method,
    'clustering_metric':metric,
    'clustering_min_samples':min_samples,
    'clustering_min_cluster_size':min_cluster_size,
    'clustering_eps':eps,
    'clustering_metric_V':V,
    'graph_generation_max_distance':max_distance,
    'graph_generation_max_angle':max_angle
}
network.set_hyperparameters(params)

In [12]:
# save network as pickle object
fileObj = open('../../data/interim/'+data_date+'_waypoints_DP'+str(tolerance)+'_HDBSCAN'+str(min_samples)+'_'+location+'_'+datasize+'_UTM.obj', 'wb')
pickle.dump(network, fileObj)
fileObj.close()